<a href="https://colab.research.google.com/github/joyalkenus/Project_imageSearch/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is the major run file of this project
- Aims to give a user friendly interface to interact with the system
- aims to integrate the BERT LLM to the RAM model to achieve proper image search for the user description.

In [20]:
#@title Import dependencies
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!pip install transformers

In [25]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [28]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap

clear_output()

In [29]:
# Download checkpoints
model_widget = widgets.Dropdown(
    options=["RAM", "Tag2Text"],
    value="RAM",
    description="Select mdoel:"
)
display(model_widget)

Dropdown(description='Select mdoel:', options=('RAM', 'Tag2Text'), value='RAM')

In [30]:
model = model_widget.value

In [31]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM
--2024-01-27 06:25:43--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 18.238.49.70, 18.238.49.112, 18.238.49.10, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.70|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-01-27 06:25:43--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_1

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def encode_text(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=tokenizer.model_max_length)
    with torch.no_grad():
        output = model(**encoded_input)
    return output.last_hidden_state.mean(dim=1)



Use the Bert model to the capion data set



In [32]:
df = !python extract.py
df


['/encoder/layer/0/crossattention/self/query is tied',
 '/encoder/layer/0/crossattention/self/key is tied',
 '/encoder/layer/0/crossattention/self/value is tied',
 '/encoder/layer/0/crossattention/output/dense is tied',
 '/encoder/layer/0/crossattention/output/LayerNorm is tied',
 '/encoder/layer/0/intermediate/dense is tied',
 '/encoder/layer/0/output/dense is tied',
 '/encoder/layer/0/output/LayerNorm is tied',
 '/encoder/layer/1/crossattention/self/query is tied',
 '/encoder/layer/1/crossattention/self/key is tied',
 '/encoder/layer/1/crossattention/self/value is tied',
 '/encoder/layer/1/crossattention/output/dense is tied',
 '/encoder/layer/1/crossattention/output/LayerNorm is tied',
 '/encoder/layer/1/intermediate/dense is tied',
 '/encoder/layer/1/output/dense is tied',
 '/encoder/layer/1/output/LayerNorm is tied',
 '--------------',
 'pretrained/ram_swin_large_14m.pth',
 '--------------',
 'load checkpoint from pretrained/ram_swin_large_14m.pth',
 'vit: swin_l',
 '  Filename  .

In [40]:
df[1]

'/encoder/layer/0/crossattention/self/key is tied'